In [ ]:
from keras.utils import Sequence
import os
import glob
import math
import numpy as np

batch_size = 16
test_images = glob.glob('../input/severstal-steel-defect-detection/test_images/*')


def msk_encode(msk):
    if np.max(msk) == 0:
        return np.nan
    msk = msk.flatten(order='F')
    msk = np.diff(msk, prepend=0, append=0)
    start, = np.where(msk == 1)
    length, = np.where(msk == -1)
    length -= start
    start += 1
    ecd = np.stack([start, length], axis=0).flatten(order='F')
    return ' '.join(map(str, ecd))


class TestDataFeeder(Sequence):

    def __init__(self):
        self.testset_dirs = test_images
        self.batch_size = batch_size
        self.length = len(self.testset_dirs)

    def __getitem__(self, index):
        import matplotlib.pyplot as plt
        import cv2
        i_start = index * self.batch_size
        i_end = min(self.length, (index + 1) * self.batch_size)
        x1, x2, y = [], [], []
        for i in range(i_start, i_end):
            y_i = self.testset_dirs[i]
            x1_i = plt.imread(y_i).copy().astype(np.float32)
            x2_i = cv2.resize(x1_i, dsize=(0, 0), fx=0.5, fy=0.5)

            x1_i -= np.mean(x1_i, keepdims=True)
            x1_i /= (np.std(x1_i, keepdims=True) + 1e-6)

            x2_i -= np.mean(x2_i, keepdims=True)
            x2_i /= (np.std(x2_i, keepdims=True) + 1e-6)

            x1.append(x1_i)
            x2.append(x2_i)

            y_i = os.path.split(y_i)[1]
            y.append(y_i)
        return [np.stack(x1, axis=0), np.stack(x2, axis=0)], y

    def __len__(self):
        return math.ceil(self.length / self.batch_size)

In [ ]:
from multiprocessing import Process, Queue


def pre_processing_worker(q, num, total):
    feeder = TestDataFeeder()
    print('worker', num + 1, 'of', total, 'started!')
    for i in range(num, len(feeder), total):
        q.put(feeder[i], block=True)
    del feeder


input_queue = Queue(maxsize=32)
worker_num = 1

workers = [
    Process(
        target=pre_processing_worker,
        args=(input_queue, i, worker_num)
    )
    for i in range(worker_num)
]
for w in workers:
    w.start()

In [ ]:
from keras.models import *
from keras.layers import *
import tensorflow as tf
import keras.backend as K
from keras.initializers import Initializer


class Swish(Layer):
    def call(self, inputs):
        return tf.nn.swish(inputs)


class DropConnect(Layer):
    def __init__(self, drop_connect_rate=0.0, **kwargs):
        super().__init__(**kwargs)
        self.drop_connect_rate = drop_connect_rate

    def call(self, inputs, training=None):
        def drop_connect():
            keep_prob = 1.0 - self.drop_connect_rate

            # Compute drop_connect tensor
            batch_size = tf.shape(inputs)[0]
            random_tensor = keep_prob
            random_tensor += tf.random_uniform(
                [batch_size, 1, 1, 1], dtype=inputs.dtype
            )
            binary_tensor = tf.floor(random_tensor)
            output = tf.div(inputs, keep_prob) * binary_tensor
            return output

        return K.in_train_phase(drop_connect, inputs, training=training)

    def get_config(self):
        config = super().get_config()
        config["drop_connect_rate"] = self.drop_connect_rate
        return config


class EfficientConv2DKernelInitializer(Initializer):
    def __call__(self, shape, dtype=K.floatx(), **kwargs):
        kernel_height, kernel_width, _, out_filters = shape
        fan_out = int(kernel_height * kernel_width * out_filters)
        return tf.random_normal(
            shape, mean=0.0, stddev=np.sqrt(2.0 / fan_out), dtype=dtype
        )


dmy = 'binary_crossentropy'

mdlcls = load_model(
    '../input/steelclsv2/weights.09-0.97.hdf5',
    custom_objects={
        'Swish': Swish,
        'DropConnect': DropConnect,
        'EfficientConv2DKernelInitializer': EfficientConv2DKernelInitializer,
        'tp1': dmy, 'tp2': dmy, 'tp3': dmy, 'tp4': dmy,
        't1': dmy, 't2': dmy, 't3': dmy, 't4': dmy,
        'p1': dmy, 'p2': dmy, 'p3': dmy, 'p4': dmy,
        'tp': dmy, 'p': dmy, 't': dmy,
        'focal': dmy
    }
)
mdlseg = load_model(
    '../input/steelseg/weights.20-4.48.hdf5',
    custom_objects={
        'Dice_CE_FPsup_Loss': dmy, 'GDL': dmy,
        'dice': dmy, 'I': dmy, 'C': dmy, 'T': dmy,
        'da': dmy, 'pa': dmy,
        'd1': dmy, 'p1': dmy, 'd2': dmy, 'p2': dmy,
        'd3': dmy, 'p3': dmy, 'd4': dmy, 'p4': dmy
    }
)

inputcls = mdlcls.layers[0].input
outputcls = mdlcls.layers[-1].output
for l in mdlcls.layers:
    l.name = 'cls_' + l.name
outputcls = Reshape(target_shape=(1, 1, 4))(outputcls)

inputseg = mdlseg.layers[0].input
outputseg = mdlseg.layers[-1].output
for l in mdlseg.layers:
    l.name = 'seg_' + l.name
outputseg = Lambda(lambda x: x[:, :, :, 1:], output_shape=(256, 1600, 4))(outputseg)
outputs = Multiply()([outputcls, outputseg])

mdl = Model(input=[inputseg, inputcls], outputs=outputs)

In [ ]:
from tqdm import trange
import pandas as pd

submission_list = []

for i in trange(math.ceil(len(test_images) / batch_size)):
    x, x_id = input_queue.get(block=True, timeout=100)
    y = mdl.predict(
        x=x,
        batch_size=batch_size,
        verbose=0
    )
    for j, x_j_id in enumerate(x_id):
        for k in range(1, 5):
            msk = np.round(y[j, :, :, k - 1])
            ep = msk_encode(msk)
            submission_list.append({'ImageId_ClassId': x_j_id + '_' + str(k), 'EncodedPixels': ep})

submission_df = pd.DataFrame(columns=['ImageId_ClassId', 'EncodedPixels'])
submission_df = submission_df.append(submission_list).set_index('ImageId_ClassId').sort_index()
submission_df.to_csv('submission.csv')

for w in workers:
    w.join()

print(submission_df)